# Содержание

* [1 Подготовка окружения](#1-Подготовка-окружения)
* [2 Mona Liza reconstruction](#2-Mona-Liza-reconstruction)

# Лабораторная работа №3: Дифференциальный рендеринг

**Задачи:** 

1. Воспроизвести эксперимент с Мона Лизой по статье [3D ML. Часть 4: дифференциальный рендеринг](https://habr.com/ru/companies/itmai/articles/520268/).
2. Произвести тот же эксперимент на своём лице (подогнать фото по размерам изображения с Мона Лизой и переложить на чёрный фон).

**Источники данных:** 1. [Basel face model (2017 version)](https://faces.dmi.unibas.ch/bfm/bfm2017.html)

**Описание данных:** изображения.

<div style="background-color: blue; height: 2px; margin: 10px 0;"></div>

## 1. Подготовка окружения

[Installation](https://github.com/BachiLi/redner/wiki/Installation)

In [19]:
%%capture no-display
!pip install redner-gpu h5py urllib3

Импорт библиотек:

In [20]:
import torch
import pyredner
import h5py
import urllib
import time

from matplotlib.pyplot import imshow
%matplotlib inline

import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from matplotlib import animation

from IPython.display import HTML

ModuleNotFoundError: No module named 'pyredner'

In [3]:
import warnings

warnings.filterwarnings('ignore')

<div style="background-color: blue; height: 2px; margin: 10px 0;"></div>

## 2 Mona Liza reconstruction

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.device(device)

print('Current device:', device)

Current device: cuda


In [5]:
# Load the Basel face model
with h5py.File(r'./models/model2017-1_bfm_nomouth.h5', 'r') as hf:
    
    shape_mean = torch.tensor(
        hf['shape/model/mean'], 
        device = device
    )
    
    shape_basis = torch.tensor(
        hf['shape/model/pcaBasis'], 
        device = device
    )
    
    triangle_list = torch.tensor(
        hf['shape/representer/cells'], 
        device = device
    )
    
    color_mean = torch.tensor(
        hf['color/model/mean'], 
        device = device
    )
    
    color_basis = torch.tensor(
        hf['color/model/pcaBasis'], 
        device = device
    )

In [10]:
indices = triangle_list.permute(1, 0).contiguous()

def model(cam_pos, 
          cam_look_at, 
          shape_coeffs, 
          color_coeffs, 
          ambient_color, 
          dir_light_intensity):
    
    vertices = (shape_mean + shape_basis @ shape_coeffs).view(-1, 3)
    normals = pyrender.compute_vertex_normal(vertices, indices)
    colors = (color_mean + color_basis @ color_coeffs).view(-1, 3)
    
    m = pyrender.Material(use_vertex_color = True)
    
    obj = pyrender.Object(
        vertices=vertices, 
        indices=indices, 
        normals=normals, 
        material=m, 
        colors=colors
    )
    
    cam = pyrender.Camera(
        position=cam_pos,
        # Center of the vertices                          
        look_at=cam_look_at,
        up=torch.tensor([0.0, 1.0, 0.0]),
        fov=torch.tensor([45.0]),
        resolution=(256, 256)
    )
    
    scene = pyrender.Scene(camera=cam, objects=[obj])
    
    ambient_light = pyrender.AmbientLight(ambient_color)
    
    dir_light = pyrender.DirectionalLight(
        torch.tensor([0.0, 0.0, -1.0]), 
        dir_light_intensity
    )
    
    img = pyrender.render_deferred(
        scene=scene, 
        lights=[ambient_light, dir_light]
    )
    
    return img

In [11]:
cam_pos = torch.tensor([-0.2697, -5.7891, 373.9277])
cam_look_at = torch.tensor([-0.2697, -5.7891, 54.7918])

img = model(
    cam_pos, 
    cam_look_at, 
    torch.zeros(199, device=device),
    torch.zeros(199, device=device),
    torch.ones(3), 
    torch.zeros(3)
)

imshow(torch.pow(img, 1.0/2.2).cpu())

face_url = 'https://raw.githubusercontent.com/BachiLi/redner/master/tutorials/mona-lisa-cropped-256.png'

urllib.request.urlretrieve(face_url, 'target.png')
target = pyrender.imread('target.png').to(pyrender.get_device())

imshow(torch.pow(target, 1.0/2.2).cpu())

AttributeError: module 'pyrender' has no attribute 'compute_vertex_normal'

---

### 2.1 Воксели

---

### 2.2 Облако точек

---

### 2.3 Функциональная модель

---

### 2.4 Карта глубины

<div style="background-color: blue; height: 2px; margin: 10px 0;"></div>

## 3. Общий вывод

<div style="text-align: center; font-size: 20px; padding: 15px 0;">
    <a href="#Содержание" data-toc-modified-id="Содержание" style="text-decoration: none; color: #296eaa; border: 2px dashed #296eaa; opacity: 0.8; border-radius: 3px; padding: 10px 80px;">
        В начало файла ↑
    </a>
</div>